# An EXP simulation using pyEXP from a notebook: a disk-halo simulation

This notebook uses `k3d` for three-dimensional respresentation of contours.  If you do not have `k3d`,  you will need to `pip install k3d` if you would like to see the three-dimensional views.  The Docker container has `k3d` preinstalled.

We begin by importing `pyEXP` and friends and setting the working directory.

In [1]:
import os
import yaml
import pyEXP
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 9]

os.chdir('DiskHalo')

# Print the current working directory for reference
print(os.getcwd())

/home/weinberg/Projects/pyEXP-examples/Tutorials/EXP/DiskHalo


## Running EXP

If you have a full EXP install or are using the Docker image, you can run EXP from inside Jupyter to generate new simulations.  We'll do
and example here using the periodic cube.

### Generating initial conditions

The full install has a body generator for the galaxy initial conditions called `gendisk`.   This code uses an Eddington inversion to
generate spherical halo initial condtions and Jeans equations to generate disk initial conditions.  It will generate the cylindrical
basis at the same time.  This step takes some time, so please be prepared to wait for 10 or 15 minutes.  Try it as follows:

In [ ]:
! mpirun gendisk --ASCALE=0.01 --HSCALE=0.002 --scale_length=0.01 --scale_height=0.002 --LMAX=2 --MMAX=2 --NMAXH=6 --NMAXD=8 --NODD=2 --RMIN=0.0001 --nhalo=60000 --ndisk=20000 --ignore --cachefile=eof.cache.test

--------------------------------------------------------------------------
btl_vader_single_copy_mechanism MCA variable, but CMA support is
not available due to restrictive ptrace settings.

The vader shared memory BTL will fall back on another single-copy
mechanism if one is available. This may result in lower performance.

  Local host: f9b35aac147f
--------------------------------------------------------------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%% Node, process, and communicator assignment %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  Node # |             Hostname |      PID |     Status
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
       0 |         f9b35aac147f |     1780 |       ROOT
       1 |         f9b35aac147f |     1783 |    WORKER 0
       2 |         f9b35aac147f |     1786 |    WORKER 1
       3 |         f9


A few comments on `gendisk` parameters used above:
- `ASCALE` is the scale length for the basis function construction
- `HSCALE` is the scale height for the basis function construction
- `scale_length` is the scale length for the particle distribution
- `scale_height` is the scale height for the particle distribution
- `LMAX` is the maximum spherical order for the halo
- `MMAX` is the maximum azimuthal order for the disk
- `NMAXH` is the maximum radial order for the halo
- `NMAXD` is the maximum radial order for the disk
- `NODD` is the number of vertically antisymmetric functions per azimuthal order
- `RMIN` is the minimum halo radius for internal tabulation
- `nhalo` is the number of desired halo particles
- `ndisk` is the number of desired disk particles
- `ignore` if true, disk basis cache file is recomputed and rewritted, even if it exists
- `cachefile` is the name of the disk basis cache file

It almost always makes sense to set the basis and distribution scale lengths and heights to the same values.  But you have the 
option to vary those.  You can view all of the possible parameters using `gendisk -h`, but these are the most often used ones and
the rest of sensible defaults for many cases.

### Running the simulation

We have provided a sample `exp` configuration file in this working directory.  The default name for a configuration file is
`config.yml`.  This run will take under an hour; and as little as 10 minutes on a modern laptop.  So be prepared to take a short
break.  For maximum speed change `X` in the `-np X` argument to `mpirun` to the number of cores available in your CPU.

The first shell command below empties the data directory, and the next is the`exp` launch command. The `exp` code will run for
500 steps.  You can estimate the total run time by looking at the last line of every step diagnostic stanza that estimates the
time needed for each part of the caluculation.  The 'Total' field is the total run time in seconds per step.  E.g. if your computer
is requiring 3 seconds per step, the completion time will be 25 minutes.

In [ ]:
%%time
! rm data/*
! mpirun exp


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.3%
--- Adj/Tot = 1.2%
--- Update  = 9.8817e-03 sec
--- Collate = 1.0700e-04 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.407124e-02     > 2.000000e-03     [41880]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41880/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.006] ---------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000679069        [  0.06 %]
Velocity            : 0.00312527         [  0.26 %]
Force              


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.4%
--- Adj/Tot = 1.4%
--- Update  = 8.2789e-03 sec
--- Collate = 9.0633e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.406686e-02     > 2.000000e-03     [41810]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41810/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.018] ---------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000850232        [  0.07 %]
Velocity            : 0.00399054         [  0.31 %]
Force              


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.7%
--- Adj/Tot = 1.3%
--- Update  = 7.9971e-03 sec
--- Collate = 7.3019e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.404666e-02     > 2.000000e-03     [41807]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41807/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.03] ----------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000864963        [  0.07 %]
Velocity            : 0.00369417         [  0.31 %]
Force              


----------------------------------------------------------------------
--- Timer info [T=0.04] ----------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000891268        [  0.07 %]
Velocity            : 0.00371508         [  0.28 %]
Force               : 0.424954           [ 31.93 %]
Coefs               : 0.15773            [ 11.85 %]
Output              : 0.10045            [  7.55 %]
Levels              : 0.000456962        [  0.03 %]
Report              : 1.726e-06          [  0.00 %]
Balance             : 6.77e-07           [  0.00 %]
Adjust              : 0.642113           [ 48.25 %]
Total               : 1.33081            [100.00 %]
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.3%
--- Adj/Tot = 1.1%
--- Update  = 8.1831e-03 sec
--- Collate = 7.2639e-05 se


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.1%
--- Adj/Tot = 1.2%
--- Update  = 7.9165e-03 sec
--- Collate = 6.8103e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=3.026128e-06     < 7.812500e-06     [1]
--- Max DT=2.404592e-02     > 2.000000e-03     [41852]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 1/20000
         Component <dark halo> |  high: 41852/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.052] ---------------------------------------------
-----------------------------------------------

...


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.7%
--- Adj/Tot = 1.2%
--- Update  = 7.7367e-03 sec
--- Collate = 6.5963e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=3.590798e-06     < 7.812500e-06     [1]
--- Max DT=2.375159e-02     > 2.000000e-03     [41682]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 1/20000
         Component <dark halo> |  high: 41682/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.958] ---------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.3%
--- Adj/Tot = 1.2%
--- Update  = 8.4651e-03 sec
--- Collate = 8.0365e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=3.028667e-06     < 7.812500e-06     [2]
--- Max DT=2.375300e-02     > 2.000000e-03     [41707]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 2/20000
         Component <dark halo> |  high: 41707/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.97] ----------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Timer info [T=0.98] ----------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000847899        [  0.06 %]
Velocity            : 0.00373671         [  0.27 %]
Force               : 0.488986           [ 35.20 %]
Coefs               : 0.181377           [ 13.06 %]
Output              : 0.0936254          [  6.74 %]
Levels              : 0.000465564        [  0.03 %]
Report              : 1.54e-06           [  0.00 %]
Balance             : 5.1e-07            [  0.00 %]
Adjust              : 0.619573           [ 44.60 %]
Total               : 1.38906            [100.00 %]
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.6%
--- Adj/Tot = 1.1%
--- Update  = 8.0586e-03 sec
--- Collate = 7.5188e-05 se


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.0%
--- Adj/Tot = 1.2%
--- Update  = 8.2833e-03 sec
--- Collate = 7.8994e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=3.710059e-06     < 7.812500e-06     [1]
--- Max DT=2.373641e-02     > 2.000000e-03     [41665]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 1/20000
         Component <dark halo> |  high: 41665/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=0.992] ---------------------------------------------
-----------------------------------------------

CPU times: user 21.7 s, sys: 3.85 s, total: 25.6 s
Wall time: 10min 53s


Restart the run to get a longer time series

In [ ]:
! mpirun exp


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.1%
--- Adj/Tot = 1.2%
--- Update  = 3.5514e-03 sec
--- Collate = 2.7109e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=2.705597e-06     < 7.812500e-06     [2]
--- Max DT=2.372542e-02     > 2.000000e-03     [41602]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 2/20000
         Component <dark halo> |  high: 41602/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.002] ---------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 7.0%
--- Adj/Tot = 1.5%
--- Update  = 8.4342e-03 sec
--- Collate = 6.5890e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=3.647204e-06     < 7.812500e-06     [1]
--- Max DT=2.373543e-02     > 2.000000e-03     [41593]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 1/20000
         Component <dark halo> |  high: 41593/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.014] ---------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.4%
--- Adj/Tot = 1.3%
--- Update  = 8.4103e-03 sec
--- Collate = 7.0034e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.372323e-02     > 2.000000e-03     [41572]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41572/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.026] ---------------------------------------------
----------------------------------------------------------------------
Drift               : 0.00178575         [  0.13 %]
Velocity            : 0.00865475         [  0.62 %]
Force              


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.7%
--- Adj/Tot = 1.3%
--- Update  = 8.6081e-03 sec
--- Collate = 9.2194e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.373309e-02     > 2.000000e-03     [41597]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41597/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.038] ---------------------------------------------
----------------------------------------------------------------------
Drift               : 0.00192043         [  0.14 %]
Velocity            : 0.00885833         [  0.65 %]
Force              


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.2%
--- Adj/Tot = 1.3%
--- Update  = 7.3377e-03 sec
--- Collate = 6.4213e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Max DT=2.372361e-02     > 2.000000e-03     [41593]
----------------------------------------------------------------------
         Component <dark halo> |  high: 41593/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.05] ----------------------------------------------
----------------------------------------------------------------------
Drift               : 0.0017878          [  0.13 %]
Velocity            : 0.00851408         [  0.64 %]
Force              

...


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.0%
--- Adj/Tot = 1.4%
--- Update  = 8.0484e-03 sec
--- Collate = 5.8835e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=2.884517e-06     < 7.812500e-06     [3]
--- Max DT=2.315657e-02     > 2.000000e-03     [40997]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 3/20000
         Component <dark halo> |  high: 40997/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.946] ---------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.3%
--- Adj/Tot = 1.3%
--- Update  = 7.6680e-03 sec
--- Collate = 5.8840e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=2.912632e-06     < 7.812500e-06     [1]
--- Max DT=2.316318e-02     > 2.000000e-03     [41042]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 1/20000
         Component <dark halo> |  high: 41042/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.958] ---------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 6.0%
--- Adj/Tot = 1.4%
--- Update  = 7.5909e-03 sec
--- Collate = 5.1220e-05 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=2.936996e-06     < 7.812500e-06     [2]
--- Max DT=2.315271e-02     > 2.000000e-03     [41031]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 2/20000
         Component <dark halo> |  high: 41031/60000
         Component <star disk> |  high: 0/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.97] ----------------------------------------------
-----------------------------------------------


----------------------------------------------------------------------
--- Timer info [T=1.98] ----------------------------------------------
----------------------------------------------------------------------
Drift               : 0.000799503        [  0.06 %]
Velocity            : 0.00428042         [  0.32 %]
Force               : 0.492272           [ 37.03 %]
Coefs               : 0.179158           [ 13.48 %]
Output              : 0.0834775          [  6.28 %]
Levels              : 0.000334519        [  0.03 %]
Report              : 1.005e-06          [  0.00 %]
Balance             : 5.07e-07           [  0.00 %]
Adjust              : 0.568724           [ 42.78 %]
Total               : 1.32945            [100.00 %]
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.7%
--- Adj/Tot = 1.2%
--- Update  = 7.4857e-03 sec
--- Collate = 7.2264e-05 se


----------------------------------------------------------------------
--- Coefficient adjust stats
--- Coef/DT = 5.9%
--- Adj/Tot = 1.3%
--- Update  = 8.0993e-03 sec
--- Collate = 1.1811e-04 sec
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Multistepping overrun
--- Min DT=1.280473e-06     < 7.812500e-06     [4]
--- Max DT=2.313917e-02     > 2.000000e-03     [41012]
----------------------------------------------------------------------
         Component <dark halo> |   low: 0/60000
         Component <star disk> |   low: 4/20000
         Component <dark halo> |  high: 41011/60000
         Component <star disk> |  high: 1/20000
----------------------------------------------------------------------

----------------------------------------------------------------------
--- Timer info [T=1.992] ---------------------------------------------
-----------------------------------------------

## Read the log file

and plot the virial equilibrium as a function of time.  The log file records the center of mass, center of velocity, angular momentum,
and energies for the total and for each component separately.  Here, we will plot the virial statistic as a diagnostic for equilibrium.

In [ ]:
!pwd
!ls data/OUTLOG.run0

In [ ]:
# This parses the  OUTLOG files generated by exp.  You can see available columns by printing the keys
# in the dictionary 'data'.  For this plot, we will only look at twice the ratio of kinetic energy to potential energy
# (estimated by the virial of Clausius).

def parseLog(runtag):
    path = 'data/OUTLOG.' + runtag
    file = open(path)
    # Get the column names
    labs = file.readlines()[2:3][0].split('|')
    # Get the data
    vals = np.loadtxt("data/OUTLOG." + runtag, delimiter='|', skiprows=6)
    # Make a dictionary of columns
    data = {}
    for i in range(len(labs)):
        data[labs[i].strip()] = vals[:,i]
    return data

# Get the dictionary
data = parseLog('run0')

# Make the plot
plt.plot(data['Time'], data['2T/VC'], label='Total')
plt.plot(data['Time'], data['star disk 2T/VC'], label = 'star disk')
plt.plot(data['Time'], data['dark halo 2T/VC'], label = 'dark halo')
plt.legend()
plt.tight_layout()
plt.xlabel('Time')
plt.ylabel('2T/VC')
plt.show()

## Reading the disk coefficients
This is a test of the coefficients created by the `EXP-examples/DiskHalo` run.

In [ ]:
# Reread the coefs from the file.  These are new HDF5 coefficients sets.
#
coefs = pyEXP.coefs.Coefs.factory('data/outcoef.star disk.run0')

print("Got coefs for name=", coefs.getName())

### Now get the time vector and data tensor. Plot some coefficient series as a function of time.

In [ ]:
times = coefs.Times()
data  = coefs.getAllCoefs()

print("The data sizes and coefficient orders are:")
print("M orders, N orders, Times:", data.shape)

### Finally, plot some coefficient values...

In [ ]:
for m in range(1, data.shape[0]):
    for n in range(4):
        plt.plot(times, np.abs(data[m, n, :]), label='{} {}'.format(m, n))

plt.xlabel('time')
plt.ylabel(r'$C_{mn}$')
plt.legend()
plt.show()

## Try loading EXP config with pyYAML

In [ ]:
with open('config.yml', 'r') as f:
    yaml_db = yaml.load(f, Loader=yaml.FullLoader)

# The 0 index force is the halo and the 1 index force is the disk, e.g.:
for i in range(2):
    print('Index {} is {}'.format(i, yaml_db['Components'][i]['name']))

config = yaml.dump(yaml_db['Components'][1]['force'])
        
print('\nIndex 1 config is:', config)

## Construct the cylindrical basis

In [ ]:
basis = pyEXP.basis.Basis.factory(config)

In [ ]:
times = coefs.Times()

In [ ]:
# Slice through the disk midplane
rmin  = -0.04
rmax  = 0.04
pmin  = [ rmin, rmin, 0]
pmax  = [ rmax, rmax, 0]
grid  = [   40,    40,  0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

The following routine rounds a value up in leading signficant figure (or down if the value is negative).  Just a
useful utility for setting contour levels on _nice_ numbers.

In [ ]:
import math

def getLim(x):
    """ Get the one signifcant figure upper/lower limit """
    om = 10.0**np.floor(np.log10(np.abs(x)))
    if x < 0.0:
        return (np.floor(x/om)*om, om)
    else:
        return (np.ceil(x/om)*om, om)

In [ ]:
densmin =  1.0e30
densmax = -1.0e30

for v in surfaces:
    for k in surfaces[v]: 
        if k=='dens':
            densmin = min(densmin, np.min(surfaces[v]['dens']))
            densmax = max(densmax, np.max(surfaces[v]['dens']))

print('Density range', densmin, densmax)

## Render the disk plane density slice

This next cell will write images to disk for making a movie.  It will take a few minutes to complete.

In [ ]:
from matplotlib import cm, ticker

# Get the evaluation grid
dr = (rmax-rmin)/(40-1)
rr = np.arange(rmin, rmax+0.1*dr, dr)
xv, yv = np.meshgrid(rr, rr)
cnt = 0

# Round to one sig figfor v in surfaces:
dmax, ommax = getLim(math.log10(densmax))

# Make levels
levs = 10.0**np.arange(dmax-4, dmax+0.01, 0.25)

for v in times:
    mat = surfaces[v]['dens']
    mat[mat<levs[0]] = levs[0]
    cc = plt.contourf(xv, yv, mat.transpose(), levels=levs, locator=ticker.LogLocator())
    plt.colorbar(cc)
    plt.title('Time={:4.3f}'.format(v))
    plt.savefig('disk_{:04d}.png'.format(cnt))
    plt.close()
    cnt += 1

#### Make a movie from the rendered frames

This requires `ffmpeg` to be installed.  It *is* installed in the Docker image...

In [ ]:
os.system('ffmpeg -y -i \'disk_%04d.png\' disk_{0}.mp4'.format('run0'))

#### Play the movie

In [ ]:
from IPython.display import Video
Video('disk_{}.mp4'.format('run0'), embed=True, width=800)

### A few comments

This simulation was restricted low harmonic and radial orders for speed and owing to the small particle number.  Feel free to regenerate
the initial conditions with larger particle numbers and increase the number of basis function expansion terms to experiment with this.
It is best although not strictly necessary to match the maximum order chosen in the initial condition generation to the orders in
the `config.yml` file used to configure the simulation.

You may have noticed from the coefficient plot and the movie that the dipole component is prominent: the disk is affected by the dipole of the halo.  It is interesting to rerun the halo without the odd-order l harmonics.

### A 3d rendering for fun

Make the volume slices for the disk component

In [ ]:
import k3d
from k3d import matplotlib_color_maps

# Slice through the cube perpendicular to the z axis
rmin  = -0.04
rmax  = 0.04
hmin  = -0.005
hmax  =  0.005
pmin  = [ rmin, rmin, hmin]
pmax  = [ rmax, rmax, hmax]
grid  = [   40,   40,   40]

# Only render the final time
fields = pyEXP.field.FieldGenerator(times[-1:], pmin, pmax, grid)

volumes = fields.volumes(basis, coefs)

Take the log of the density field and find some good contour levels

In [ ]:
# levels for rendering 
dmax, ommax = getLim(math.log10(densmax))
llevs = np.arange(dmax-2, dmax+0.01, 0.5)
levs = 10.0**llevs
print("Level surfaces:", levs, llevs)

v = volumes[times[-1]]['dens']
v[v<levs[0]] = 0.5*levs[0]
v = np.log10(v)

Finally, make the 3d rendering using `k3d`

In [ ]:
plot = k3d.plot(axes=['z', 'y', 'x'])
color_map = k3d.basic_color_maps.Jet

minlev = dmax-2.0
maxlev = dmax

for lev in llevs:
    plot += k3d.marching_cubes(v, level=lev, attribute=[lev], color_map=color_map, color_range=[minlev, maxlev],
                            opacity=0.25,
                            xmin=hmin, xmax=hmax,
                            ymin=rmin, ymax=rmax,
                            zmin=rmin, zmax=rmax,
                            compression_level=9,
                            flat_shading=False)
plot.display()
plot.camera = [0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 1, 0, 0]



### Visualizing the halo wake

We can repeat the same rendering for the halo to see the wake induced by the bar!

First, we need to create the halo basis and read the halo coefficients.

In [ ]:
halo_config = yaml.dump(yaml_db['Components'][0]['force'])
halo_basis = pyEXP.basis.Basis.factory(halo_config)
halo_coefs = pyEXP.coefs.Coefs.factory('data/outcoef.dark halo.run0')

In [ ]:
coef_data = halo_coefs.getAllCoefs()

# Let's zero all of the odd order data
for k in range(coef_data.shape[0]):
    l, m = pyEXP.basis.SphericalSL.invI(k)
    if l%2!=0 or m%2!=0: coef_data[k,:,:] *= 0.0

# Reset the coefficient data
for i in range(coef_data.shape[2]):
    halo_coefs.setMatrix(times[i], coef_data[:,:,i])

Now, we can render the volume resconstruction for a cube around the disk with same extent as the disk rendering.

In [ ]:
rmin  = -0.04
rmax  =  0.04
pmin  = [ rmin, rmin, rmin]
pmax  = [ rmax, rmax, rmax]
grid  = [   40,   40,   40]

# Only render the final time
halo_fields = pyEXP.field.FieldGenerator(times[-1:], pmin, pmax, grid)

halo_volumes = halo_fields.volumes(halo_basis, halo_coefs)

To emphasize the wake, we'll render the non-spherical components only

In [ ]:
# Get the non-axisymmetric density

v = halo_volumes[times[-1]]['dens m>0']
dmin = np.min(v)
dmax = np.max(v)

print('Density range', dmin, dmax)

# levels for rendering 
drange = (dmax - dmin)/6
levs = np.arange(dmin, dmax+0.01*drange, drange)

print("Level surfaces:", levs)

plot2 = k3d.plot(axes=['z', 'y', 'x'])
color_map = k3d.basic_color_maps.Jet

for lev in levs:
    plot2 += k3d.marching_cubes(v, level=lev, attribute=[lev], color_map=color_map, color_range=[levs[0], levs[-1]],
                            opacity=0.25,
                            xmin=rmin, xmax=rmax,
                            ymin=rmin, ymax=rmax,
                            zmin=rmin, zmax=rmax,
                            compression_level=9,
                            flat_shading=False)
plot2.display()
plot2.camera = [0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 1, 0, 0]